### Housekeeping

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from Settings import columns, experiments
import Pipeline

In [ ]:
tf.__version__
keras.__version__

In [ ]:
current_exp = experiments['all']

pipeline = Pipeline.DataClean(columns, current_exp)
pipeline.clean_data()
pipeline.norm()
pipeline.split_label()
pipeline.train_val()  

Look into just using environmental variable instead of using the previous persistence of solar IR

In [ ]:
pipeline.floatcols

In [ ]:
print(type(pipeline.xval),
pipeline.yval.shape,
pipeline.xtrain.shape,
pipeline.ytrain.shape)

### Skeleton
Below is creating the "skeleton" of the model. It does not need any specific data, only the shape and what it will do

In [ ]:
pipeline.ytrain[0].shape

In [ ]:
# There are 17 columns and there needs to be at least two hours of data input which means input shape of [180, 17]
input_layer = keras.layers.Input(shape=(pipeline.xtrain[0].shape))


# First we start with an input layer
# input_layer = Input(shape=Xtrain[0].shape) # size per sample, equal to number of features

# Let's apply dropout to the input layer. (If Dropout(0), Dropout isn't being used)
layers = keras.layers.Dropout(0)(input_layer)

# Now let's add a few layers. We are going to do two layers with 5 nodes each.
# Each uses the 'relu' activation function and randomizes the initial weights/biases
# Dropout may be useful for hidden layers if the number of hidden nodes is large
# If Dropout(0), Dropout isn't being used.

SEED = 26

layers = keras.layers.Dense(10, activation='relu',
               bias_initializer=tf.keras.initializers.RandomNormal(seed=SEED),
               kernel_initializer=tf.keras.initializers.RandomNormal(seed=SEED))(layers)

layers = keras.layers.Dropout(0)(layers)

# Output layer has a softmax function to convert output to class likelihood
output_layer = keras.layers.Dense(pipeline.ytrain[0].shape[0], activation='softmax',
                  bias_initializer=tf.keras.initializers.RandomNormal(seed=SEED),
                  kernel_initializer=tf.keras.initializers.RandomNormal(seed=SEED))(layers)



# model.add(keras.layers.Dense(300, activation=current_exp['activation']))
# model.add(keras.layers.Dense(100, activation=current_exp['activation']))
# model.add(keras.layers.Dense(10, activation=current_exp['activation']))

# Similarly 
# model = keras.models.Sequential([
#     keras.layers.Flaten(input_shape=[180,17]),
#     keras.layers.Dense(300, activation=current_exp['activation']),
#     keras.layers.Dense(100, activation=current_exp['activation']),
#     keras.layers.Dense(10, activation=current_exp['activation']),
# ])

model= keras.Model(input_layer, output_layer)

### The shapes are incompatible because last shape output is 10 and it should be 120???

Tensorflow.data
I can batch things?? Batch train. 

In [ ]:
model.summary()
# hidden1 = model.layers[1]
# weights,biases = hidden1.get_weights()
# weights  # weights randomly initialized

### Compile model


In [ ]:
model.compile(loss=keras.losses.MeanSquaredError(),
             optimizer='sgd',
             metrics=['accuracy'])

### Training and Evaluating the model

In [ ]:
history = model.fit(pipeline.xtrain, pipeline.ytrain, epochs=30, validation_data=[pipeline.xval, pipeline.yval])